In [ ]:
import pandas as pd
import numpy as np
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import torch.nn as F
from copy import copy
from tqdm.notebook import tqdm as tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from torch.utils.data import TensorDataset, DataLoader
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

**Параметры запуска:**

validate - Если true, то производится валидация на обучающей выборке, иначе вычисляется ответ на тестовой

calculate_best - Если true то подбираются лучшие гиперпараметры алгоритмов, иначе нет. Функция также требует включённой валидации.

In [ ]:
validate = True
calculate_best = False
if(calculate_best and not validate):
    print("ERROR: when calculating best parameters, validation is needed. calculate_best was switched to <False>")
    calculate_best = False

In [ ]:
data_test = pd.read_csv("covid_data_test.csv", sep=',')

In [ ]:
ids_test = data_test['Unnamed: 0']
ids_test

In [ ]:
data_test = data_test.drop(columns=['Unnamed: 0'])

## Инициализация

In [ ]:
data = pd.read_csv("covid_data_train.csv", sep=',').drop(columns='Unnamed: 0')
data = data.drop(data[data['inf_rate'].isna()].index)
data = data.set_index(np.arange(data.shape[0]))

In [ ]:
data.shape

Объединяем data и data_test для параллельной предобработки

In [ ]:
train_size = data.shape[0]
data = pd.concat([data, data_test])
data = data.set_index(np.arange(data.shape[0]))

Подгружаем данные о ВВП из дополнительной таблицы

In [ ]:
vvp = pd.read_csv("vvp.csv", sep=',').drop(columns='Unnamed: 0')

In [ ]:
vvp_features = vvp.columns.values[1:]

In [ ]:
mapp = {}
for i in range(vvp.shape[0]):
    mapp[vvp['subject'][i]] = vvp.loc[i]

In [ ]:
for feature in vvp_features:
    data[feature] = np.zeros(data.shape[0])
for i in range(data.shape[0]):
    val = mapp[data['subject'][i]]
    for feature in vvp_features:
        data[feature][i] = float(val[feature].replace(',', '.'))

Удаляем лишние столбцы (в которых много NaN, и дубликаты)

In [ ]:
data = data.drop(columns=['ivl_number', 'ekmo_number',
                   'epirank_avia', 'epirank_avia_cat', 'epirank_train_cat', 'epirank_train',
                  'ecology', 'cleanness', 'public_services', 'neighbourhood', 'children_places', 'sport_and_outdoor', 'shops_and_malls', 'public_transport', 'security', 'life_costs', 'life_quality_place_rating'])

In [ ]:
divby_total = ['work_ratio_15-72_years', 'work_ratio_55-64_years', 
       'work_ratio_15-24_years', 'work_ratio_15-64_years', 
       'work_ratio_25-54_years',
       'num_patients_tubercul_1992', 'num_patients_tubercul_1993',
       'num_patients_tubercul_1994', 'num_patients_tubercul_1995',
       'num_patients_tubercul_1996', 'num_patients_tubercul_1997',
       'num_patients_tubercul_1998', 'num_patients_tubercul_1999',
       'num_patients_tubercul_2000', 'num_patients_tubercul_2001',
       'num_patients_tubercul_2002', 'num_patients_tubercul_2003',
       'num_patients_tubercul_2004', 'num_patients_tubercul_2005',
       'num_patients_tubercul_2006', 'num_patients_tubercul_2007',
       'num_patients_tubercul_2008', 'num_patients_tubercul_2009',
       'num_patients_tubercul_2010', 'num_patients_tubercul_2011',
       'num_patients_tubercul_2012', 'num_patients_tubercul_2013',
       'num_patients_tubercul_2014', 'num_patients_tubercul_2015',
       'num_patients_tubercul_2016', 'num_patients_tubercul_2017',
       'volume_serv_household_2017', 'volume_serv_chargeable_2017',
       'volume_serv_transport_2017', 'volume_serv_post_2017',
       'volume_serv_accommodation_2017', 'volume_serv_telecom_2017',
       'volume_serv_others_2017', 'volume_serv_veterinary_2017',
       'volume_serv_housing_2017', 'volume_serv_education_2017',
       'volume_serv_medicine_2017', 'volume_serv_disabled_2017',
       'volume_serv_culture_2017', 'volume_serv_sport_2017',
       'volume_serv_hotels_2017', 'volume_serv_tourism_2017',
       'volume_serv_sanatorium_2017', 'bus_march_travel_18',
       'bus_april_travel_18']


divby_rural = ['rural_50-54_years',
       'rural_55-59_years', 'rural_60-64_years', 'rural_65-69_years',
       'rural_70-74_years', 'rural_75-79_years', 'rural_80-84_years',
       'rural_85-89_years', 'rural_90-94_years',
        'num_phones_rural_2018']
divby_urban = ['urban_50-54_years',
       'urban_55-59_years', 'urban_60-64_years', 'urban_65-69_years',
       'urban_70-74_years', 'urban_75-79_years', 'urban_80-84_years',
       'urban_85-89_years', 'urban_90-94_years',
        'num_phones_urban_2019']

Эти фичи более информативны при делении на население, поэтому поделим

In [ ]:
total_cnt = data['whole_population']
total_cnt = total_cnt.replace(to_replace=0, value=1)
rural_cnt = data['rural']
rural_cnt = rural_cnt.replace(to_replace=0, value=1)
urban_cnt = data['urban']
rural_cnt = urban_cnt.replace(to_replace=0, value=1)
data[divby_total] = data[divby_total].divide(total_cnt, axis='rows')
data[divby_rural] = data[divby_rural].divide(rural_cnt, axis='rows')
data[divby_urban] = data[divby_urban].divide(urban_cnt, axis='rows')

In [ ]:
data.shape

In [ ]:
data.select_dtypes(include=['object'])

Эти фичи неинформативны, удаляем

In [ ]:
data = data.drop(columns=['name', 'region_x'])

In [ ]:
temp = data.isna().sum()
temp[temp!=0]

In [ ]:
data.info()

Делим на train и test

In [ ]:
if(validate):
    data = data.loc[:train_size-1]
    data, data_test = train_test_split(data)
else:
    data_test = data.loc[train_size:]
    data = data.loc[:train_size-1]
    data_test = data_test.set_index(np.arange(data_test.shape[0]))

Избавляемся от NaN-ов

In [ ]:
data = data.fillna(data.mean())
data_test = data_test.fillna(data_test.mean())

In [ ]:
y = data['inf_rate']
y_test = data_test['inf_rate']
data = data.drop(columns=['inf_rate'])
data_test = data_test.drop(columns=['inf_rate'])

Стандартизуем данные

In [ ]:
scaler = StandardScaler()
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
scaler.fit(data.select_dtypes(include=numerics))
data[data.select_dtypes(include=numerics).columns] = scaler.transform(data.select_dtypes(include=numerics))
data_test[data_test.select_dtypes(include=numerics).columns] = scaler.transform(data_test.select_dtypes(include=numerics))

### Удаление лишних признаков

In [ ]:
best_params_catboost = {
    'depth':5,
    'iterations':2000,
    'learning_rate':0.009,
    'border_count':40,
} #параметры получены из перебора, он сам показан в следующем разделе
reg = CatBoostRegressor(random_state=42, iterations=best_params_catboost['iterations'],depth=best_params_catboost['depth'], learning_rate=best_params_catboost['learning_rate'],
                        loss_function="MAE", cat_features=data.select_dtypes(include='object').columns.values, task_type="GPU", logging_level="Silent")
reg.fit(data, y)
indexes = np.argsort(reg.feature_importances_)
features_sorted = data.columns.values[indexes[::-1]]

In [ ]:
features_sorted

Оставим самые важные 40 признаков, так как обычно избыточное их число ухудшает качество модели

In [ ]:
data = data[features_sorted[:40]]
data_test = data_test[features_sorted[:40]]

## Подбор гиперпараметров деревьев

### GradientBoostingRegressor

In [ ]:
params_gboost={
    'max_depth':[5, 6],
    'n_estimators':[1000, 1200, 1400],
    'learning_rate':[0.25, 0.3],
}
cur_params_gboost = {
    'max_depth':0,
    'n_estimators':0,
    'learning_rate':0,
}
best_params_gboost = {
    'max_depth':7,
    'n_estimators':1000,
    'learning_rate':0.3,
}
if calculate_best:
    best_score = 10000
    for cur_params_gboost['max_depth'] in params_gboost['max_depth']:
        for cur_params_gboost['n_estimators'] in params_gboost['n_estimators']:
            for cur_params_gboost['learning_rate'] in params_gboost['learning_rate']:
                reg = GradientBoostingRegressor(random_state = 42, n_estimators=cur_params_gboost['n_estimators'], max_depth=cur_params_gboost['max_depth'], learning_rate=cur_params_gboost['learning_rate'])
                reg.fit(data.select_dtypes(include=numerics), y)
                pred = reg.predict(data_test.select_dtypes(include=numerics))
                score = np.mean(np.abs(pred-y_test))
                print(cur_params_gboost, ':\t\t', score)
                if(score<best_score):
                    best_score = score
                    best_params_gboost = copy(cur_params_gboost)
    print()
    print("Best score:")
    print(best_params_gboost)
    print(best_score)

### ExtraTreesRegressor

In [ ]:
params_xtrees={
    'max_depth':[14, 15, 16],
    'n_estimators':[800, 1000, 1200],
}
cur_params_xtrees = {
    'max_depth':0,
    'n_estimators':0,
}
best_params_xtrees = {
    'max_depth':22,
    'n_estimators':1000,
}
if calculate_best:
    best_score = 10000
    for cur_params_xtrees['max_depth'] in params_xtrees['max_depth']:
        for cur_params_xtrees['n_estimators'] in params_xtrees['n_estimators']:
            reg = ExtraTreesRegressor(random_state = 42, n_estimators=cur_params_xtrees['n_estimators'],max_depth=cur_params_xtrees['max_depth'])
            reg.fit(data.select_dtypes(include=numerics), y)
            pred = reg.predict(data_test.select_dtypes(include=numerics))
            score = np.mean(np.abs(pred-y_test))
            print(cur_params_xtrees, ':\t\t', score)
            if(score<best_score):
                best_score = score
                best_params_xtrees = copy(cur_params_xtrees)
    print()
    print("Best score:")
    print(best_params_xtrees)
    print(best_score)

### RandomForestRegressor

In [ ]:
params_rforest={
    'max_depth':[11, 12, 13],
    'n_estimators':[1000, 1200, 1400],
}
cur_params_rforest = {
    'max_depth':0,
    'n_estimators':0,
}
best_params_rforest = {
    'max_depth':13,
    'n_estimators':1400,
}
if calculate_best:
    best_score = 10000
    for cur_params_rforest['max_depth'] in params_rforest['max_depth']:
        for cur_params_rforest['n_estimators'] in params_rforest['n_estimators']:
            reg = RandomForestRegressor(random_state=42, n_estimators=cur_params_rforest['n_estimators'],max_depth=cur_params_rforest['max_depth'])
            reg.fit(data.select_dtypes(include=numerics), y)
            pred = reg.predict(data_test.select_dtypes(include=numerics))
            score = np.mean(np.abs(pred-y_test))
            print(cur_params_rforest, ':\t\t', score)
            if(score<best_score):
                best_score = score
                best_params_rforest = copy(cur_params_rforest)
    print()
    print("Best score:")
    print(best_params_rforest)
    print(best_score)

### CatBoostRegressor

In [ ]:
params_catboost={
    'depth':[5, 6],
    'iterations':[2000, 2500],
    'learning_rate':[0.009, 0.01],
    'border_count':[40, 50],
}
cur_params_catboost = {
    'depth':0,
    'iterations':0,
    'learning_rate':0,
    'border_count':0,
}
best_params_catboost = {
    'depth':5,
    'iterations':2000,
    'learning_rate':0.009,
    'border_count':40,
}
if calculate_best:
    best_score = 10000
    for cur_params_catboost['depth'] in params_catboost['depth']:
        for cur_params_catboost['iterations'] in params_catboost['iterations']:
            for cur_params_catboost['learning_rate'] in params_catboost['learning_rate']:
                for cur_params_catboost['border_count'] in params_catboost['border_count']:
                    reg = CatBoostRegressor(random_state=42, iterations=cur_params_catboost['iterations'],depth=cur_params_catboost['depth'], learning_rate=cur_params_catboost['learning_rate'],
                                           loss_function="MAE", cat_features=data.select_dtypes(include='object').columns.values, task_type="GPU", logging_level="Silent")
                    reg.fit(data, y)
                    pred = reg.predict(data_test)
                    score = np.mean(np.abs(pred-y_test))
                    print(cur_params_catboost, ':\t\t', score)
                    if(score<best_score):
                        best_score = score
                        best_params_catboost = copy(cur_params_catboost)
    print()
    print("Best score:")
    print(best_params_catboost)
    print(best_score)

## Получение предсказаний из моделей

In [ ]:
reg = GradientBoostingRegressor(random_state=42, n_estimators=best_params_gboost['n_estimators'],max_depth=best_params_gboost['max_depth'], learning_rate=best_params_gboost['learning_rate'])
reg.fit(data.select_dtypes(include=numerics), y)
data_gboost = reg.predict(data.select_dtypes(include=numerics))
data_gboost_test = reg.predict(data_test.select_dtypes(include=numerics))
if validate:
    print("GradientBoostingRegressor score:", np.mean(np.abs(data_gboost_test-y_test)))

reg = ExtraTreesRegressor(random_state=42, n_estimators=best_params_xtrees['n_estimators'],max_depth=best_params_xtrees['max_depth'])
reg.fit(data.select_dtypes(include=numerics), y)
data_xtrees = reg.predict(data.select_dtypes(include=numerics))
data_xtrees_test = reg.predict(data_test.select_dtypes(include=numerics))
if validate:
    print("ExtraTreesRegressor score:", np.mean(np.abs(data_xtrees_test-y_test)))
    
reg = RandomForestRegressor(random_state=42, n_estimators=best_params_rforest['n_estimators'],max_depth=best_params_rforest['max_depth'])
reg.fit(data.select_dtypes(include=numerics), y)
data_rforest = reg.predict(data.select_dtypes(include=numerics))
data_rforest_test = reg.predict(data_test.select_dtypes(include=numerics))
if validate:
    print("RandomForestRegressor score:", np.mean(np.abs(data_rforest_test-y_test)))
    
reg = CatBoostRegressor(random_state=42, iterations=best_params_catboost['iterations'],depth=best_params_catboost['depth'], learning_rate=best_params_catboost['learning_rate'],
                                           loss_function="MAE", cat_features=data.select_dtypes(include='object').columns.values, task_type="GPU", logging_level="Silent")
reg.fit(data, y)
data_catboost = reg.predict(data)
data_catboost_test = reg.predict(data_test)
if validate:
    print("CatBoostRegressor score:", np.mean(np.abs(data_catboost_test-y_test)))

In [ ]:
data2 = np.hstack([np.reshape(data_gboost, (-1, 1)), np.reshape(data_xtrees, (-1, 1)), np.reshape(data_rforest, (-1, 1)), np.reshape(data_catboost, (-1, 1))])
data2_test = np.hstack([np.reshape(data_gboost_test, (-1, 1)), np.reshape(data_xtrees_test, (-1, 1)), np.reshape(data_rforest_test, (-1, 1)), np.reshape(data_catboost_test, (-1, 1))])

## Стекинг

Объединяем предсказания моделей при помощи ещё одной для получения лучшего результата

In [ ]:
reg = GradientBoostingRegressor(random_state=42, n_estimators=1400 ,max_depth=6, learning_rate=0.3)
reg.fit(data2, y)
if validate:
    print("Final score: ", np.mean(np.abs(reg.predict(data2_test)-y_test)))
else:
    res = reg.predict(data2_test)
    df = pd.DataFrame(data={'inf_rate':res}).set_index(ids_test)
    df.to_csv("res.csv")#Сохраняем результат